# Use case of the INCA Parser: Isotopically non-stationary 13C-MFA


In [2]:
import pandas as pd
import numpy as np
import dotenv
import ast
import pandera as pa
import BFAIR.mfa.INCA.dataschemas as dataschemas
import BFAIR.mfa.INCA.INCAScript_writing as INCAScript_writing
from BFAIR.mfa.INCA.run_inca import run_inca
from BFAIR.mfa.visualization.schema_overview import present_schema_overview
import BFAIR.mfa.utils.merge_reversible_reactions as mrr
import pathlib

In [3]:
# import environment variables
INCA_base_directory = dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory")
data_dir = pathlib.Path('./Literature data/Astrocytes and neural stem cells/')
data_dir.exists()

True

First, we will load and process the reactions data. We will start by refreshing the requirements for the dataframe. 

In [10]:
present_schema_overview(dataschemas.ReactionsSchema)

,column name,dtype,required,nullable,description
0,rxn_id,str,True,False,The unique id of the reaction
1,rxn_eqn,str,True,False,"The reaction equation with atom map. Allowed reaction arrows: ->, <->."


We will now read in the data and rename the column to fit the requirements from `ReactionsSchema`.

In [14]:
xl = pd.ExcelFile(data_dir / 'Model.xlsx')
reactions = (xl.parse('Sheet1')
    .rename(columns={'Reaction ID': 'rxn_id', 'Equations (Carbon atom transition)': 'rxn_eqn'})
)
reactions.head()

,rxn_id,rxn_eqn
0,R1,G6P (abcdef) <-> F6P (abcdef)
1,R2,F6P (abcdef) -> FBP (abcdef)
2,R3,FBP (abcdef) <-> DHAP (cba) + GAP (def)
3,R4,DHAP (abc) <-> GAP (abc)
4,R5,GAP (abc) <-> 3PG (abc)


We are now ready to validate the data. This is easiest done if the validation is wrapped in the try-except statement, because will produce a much nicer output in case the validation does not pass.

In [16]:
try:
    dataschemas.ReactionsSchema.validate(reactions)
except pa.schemas.errors.SchemaError as e:
    print(type(e))
    print(e)

The data passes the validation and we can move on to measurements of the exchange fluxes.

In [37]:
present_schema_overview(dataschemas.FluxMeasurementsSchema)

,column name,dtype,required,nullable,description
0,experiment_id,str,True,False,"ID of the experiment. Must be a valid MATLAB variable name, legal characters are a-z, A-Z, 0-9, and the underscore character."
1,rxn_id,str,True,False,The unique id of the reaction
2,flux,float64,True,False,Measured/estimated rate typically in mmol/gDW/h
3,flux_std_error,float64,True,False,Standard error of the measured/estimated rate


In [30]:
flux_measurements_raw = pd.read_csv(data_dir / 'exchange_fluxes.csv')
flux_measurements_raw.head()

,Metabolite,Neural Stem Cells,Astrocytes
0,Glucose,-256.3 ± 1.3,-175.7 ± 17.9
1,Lactate,237.8 ± 10.1,348.6 ± 9.6
2,Aspartate,-2.6 ± 0.7,-11.9 ± 5.8
3,Glutamate,-1.8 ± 0.4,-10.5 ± 3.4
4,Serine,-2.8 ± 1.4,-4.1 ± 1.7


In [56]:
flux_measurements = (flux_measurements_raw
    .melt(id_vars=['Metabolite'], value_vars=['Astrocytes', 'Neural Stem Cells'], var_name='Cell type', value_name='flux')
    .replace({'flux': {'nan': np.nan}})
)

flux_measurements[['flux', 'flux_std_error']] = flux_measurements['flux'].str.split('±', expand=True).astype(float)
flux_measurements['experiment_id'] = flux_measurements['Cell type'].str.replace(' ', '_')
flux_measurements.head()

,Metabolite,Cell type,flux,flux_std_error,experiment_id
0,Glucose,Astrocytes,-175.7,17.9,Astrocytes
1,Lactate,Astrocytes,348.6,9.6,Astrocytes
2,Aspartate,Astrocytes,-11.9,5.8,Astrocytes
3,Glutamate,Astrocytes,-10.5,3.4,Astrocytes
4,Serine,Astrocytes,-4.1,1.7,Astrocytes


The raw data uses the metabolite to reference which the exchange reactions. The INCA Parse require the reaction id (`rxn_id`) to attach a flux measurement to a reaction. Next, we make a map from metabolite to `rxn_id`.

In [57]:
exchange_reactions_metabolite_map = {
    'CO2': 'R44',
    'Glucose': 'R45',
    'Lactate': 'R46',
    'Alanine': 'R47',
    'Pyruvate': 'R48',
    'Citrate': 'R49',
    'Glutamine': 'R50',
    'Glutamate': 'R51',
    'Aspartate': 'R52',
    'Asparagine': 'R53',
    'Serine': 'R54',
    'Glycine': 'R55',
    'Proline': 'R56',
    'Valine': 'R57',
    'Isoleucine': 'R58',
    'Leucine': 'R59',
    'Threonine': 'R60',
    'Phenylalanine': 'R61',
    'Tyrosine': 'R62',
    'Methionine': 'R63',
    'Lysine': 'R64',
    'Histidine': 'R65',
    'Arginine': 'R66',
}

We can use this map to create a new colum with the reaction ids.

In [58]:
flux_measurements['rxn_id'] = flux_measurements['Metabolite'].map(exchange_reactions_metabolite_map)
flux_measurements.head()

,Metabolite,Cell type,flux,flux_std_error,experiment_id,rxn_id
0,Glucose,Astrocytes,-175.7,17.9,Astrocytes,R45
1,Lactate,Astrocytes,348.6,9.6,Astrocytes,R46
2,Aspartate,Astrocytes,-11.9,5.8,Astrocytes,R52
3,Glutamate,Astrocytes,-10.5,3.4,Astrocytes,R51
4,Serine,Astrocytes,-4.1,1.7,Astrocytes,R54


In [59]:
try:
    dataschemas.FluxMeasurementsSchema.validate(flux_measurements)
except pa.schemas.errors.SchemaError as e:
    print(type(e))
    print(e)

<class 'pandera.errors.SchemaError'>
non-nullable series 'flux' contains null values:
6    NaN
12   NaN
13   NaN
14   NaN
17   NaN
29   NaN
30   NaN
31   NaN
39   NaN
40   NaN
Name: flux, dtype: float64


Ups, the validation failed because the INCA Parser does not allow null values in the measured fluxes. In the supplementary documentation the authors write: "NaN represents undetermined transport rates". We interpret as these rate were not measured, thus we don't know anything about the rate and it could take any value. Therefore, we will simply remove the measurements with NaN. 

An alternative interpretation is that the rate were measured but below detection limits. In that case the NaNs should be replaced by small numbers. 

In [61]:
flux_measurements.dropna(subset=['flux'], inplace=True)

try:
    dataschemas.FluxMeasurementsSchema.validate(flux_measurements)
except pa.schemas.errors.SchemaError as e:
    print(type(e))
    print(e)

Now, the flux measurements dataframe passes the validation.